# 🔐 Google Authentication Setup

**Purpose**: Get Google Calendar permissions and save tokens.

**What this does**: Opens browser → Google login → Saves tokens to JSON file

> **Note**: Calendar operations (read/create events) are in `google_calendar_operations.ipynb`

## 📋 Prerequisites

1. **Google Cloud Console** → Create project → Enable "Calendar API"
2. **Create OAuth Credentials** → "Desktop application" → Copy client ID & secret
3. **Add BOTH Redirect URIs** in Google Cloud Console:
   - `http://localhost:3000/` ⚠️ **CRITICAL: Include the trailing slash!**
   - `http://localhost:3000/auth/callback`
4. **Create .env file** in this folder:
   ```
   GOOGLE_CLIENT_ID=your_client_id_here
   GOOGLE_CLIENT_SECRET=your_client_secret_here
   GOOGLE_REDIRECT_URI=http://localhost:3000/auth/callback
   ```

## 🔧 **IMPORTANT**: 
The `run_local_server()` method automatically uses `http://localhost:3000/` with a trailing slash!


In [1]:
# Install required packages
!pip install google-auth google-auth-oauthlib python-dotenv


In [2]:
import os
from dotenv import load_dotenv
from google_auth_oauthlib.flow import InstalledAppFlow

# Load environment variables
load_dotenv()

# Configuration
SCOPES = ['https://www.googleapis.com/auth/calendar']
TOKEN_FILE = 'google_tokens.json'

# Get credentials from .env
CLIENT_ID = os.getenv('GOOGLE_CLIENT_ID')
CLIENT_SECRET = os.getenv('GOOGLE_CLIENT_SECRET')
REDIRECT_URI = os.getenv('GOOGLE_REDIRECT_URI')

if CLIENT_ID and CLIENT_SECRET and REDIRECT_URI:
    print("Credentials loaded from .env")
    print(f"Using redirect URI: {REDIRECT_URI}")
else:
    print("❌ Missing credentials in .env file!")
    print("Required: GOOGLE_CLIENT_ID, GOOGLE_CLIENT_SECRET, GOOGLE_REDIRECT_URI")


Credentials loaded from .env
Using redirect URI: http://localhost:3000/auth/callback


In [3]:
def authenticate():
    """Get Google Calendar permissions and save tokens"""
    if not CLIENT_ID or not CLIENT_SECRET:
        print("❌ Cannot authenticate - missing .env credentials")
        return None
    
    try:
        print("🔍 Debugging redirect URI configuration...")
        print(f"Your .env redirect URI: {REDIRECT_URI}")
        
        # IMPORTANT: run_local_server() automatically adds trailing slash!
        actual_redirect_uri = "http://localhost:3000/"
        
        print(f"⚠️  run_local_server() will actually use: {actual_redirect_uri}")
        print("🔧 Your Google Cloud Console MUST have BOTH:")
        print(f"   ✅ {REDIRECT_URI}")
        print(f"   ❌ {actual_redirect_uri} ← ADD THIS ONE!")
        print()
        
        # Create OAuth flow with the correct redirect URI
        client_config = {
            "installed": {
                "client_id": CLIENT_ID,
                "client_secret": CLIENT_SECRET,
                "auth_uri": "https://accounts.google.com/o/oauth2/auth",
                "token_uri": "https://oauth2.googleapis.com/token",
                "redirect_uris": [actual_redirect_uri]
            }
        }
        
        flow = InstalledAppFlow.from_client_config(client_config, scopes=SCOPES)
        
        print("🌐 Opening browser for authentication...")
        credentials = flow.run_local_server(port=3000, prompt='consent')
        
        # Save tokens
        with open(TOKEN_FILE, 'w') as f:
            f.write(credentials.to_json())
        
        print(f"✅ Success! Tokens saved to {TOKEN_FILE}")
        return credentials
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\n🚨 FIX REQUIRED:")
        print("Add this EXACT redirect URI to your Google Cloud Console:")
        print("   http://localhost:3000/")
        print("   ☝️ NOTICE THE TRAILING SLASH!")
        print()
        print("Steps:")
        print("1. Go to Google Cloud Console → APIs & Services → Credentials")
        print("2. Click your OAuth 2.0 Client ID")
        print("3. Add URI: http://localhost:3000/")
        print("4. Save and try again")
        return None

# Run authentication
credentials = authenticate()


🔍 Debugging redirect URI configuration...
Your .env redirect URI: http://localhost:3000/auth/callback
⚠️  run_local_server() will actually use: http://localhost:3000/
🔧 Your Google Cloud Console MUST have BOTH:
   ✅ http://localhost:3000/auth/callback
   ❌ http://localhost:3000/ ← ADD THIS ONE!

🌐 Opening browser for authentication...
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=547915784535-fj5mm9eia5hidrk77e3eis3d0tpuh5j7.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A3000%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=v1RdgHyoBpM0OnkevYlG4L3Ns6YOfh&prompt=consent&access_type=offline
✅ Success! Tokens saved to google_tokens.json


## ✅ Done!

If successful, you now have:
- `google_tokens.json` - Your saved authentication tokens

**Next step**: Open `google_calendar_operations.ipynb` for calendar functions

**Security**: Add `google_tokens.json` and `.env` to your `.gitignore`
